In [ ]:
!pip install gigachain faiss-cpu sentence-transformers torch langchain_comunity

In [17]:
from langchain.chat_models import GigaChat
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import create_retrieval_chain
from langchain.docstore.document import Document
import json
import warnings
warnings.filterwarnings("ignore")

In [18]:
with open("rag.json", "r", encoding="utf-8") as file:
    json_data = json.load(file)

In [19]:
API_KEY = "Yjg4MTQzMmUtNDAwMS00NDk0LThjOGUtNmU5ZWQ2YzQ4NDQ2OjcxODg0YjhmLTYyMDItNDQ5ZC04NjI2LTY1NjM0ZjU1MGJlYg=="
SCOPE = "GIGACHAT_API_CORP"
MODEL = "GigaChat-Pro"
IS_VERIFY_SSL_CERTS = False
IS_PROFANITY_CHECK = False

In [20]:
documents = []
for item in json_data:
    doc = Document(
        page_content=item["text"],  
        metadata={
            "id": item["id"],
            "title": item["title"],
            "page": item["page"],
            "keywords": item["keywords"]
        }
    )
    documents.append(doc)

In [21]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
)
documents = text_splitter.split_documents(documents)
print(f"Total documents: {len(documents)}")

Total documents: 58


In [22]:
documents[30]

Document(metadata={'id': '28', 'title': 'Портированная версия в PL/pgSQL', 'page': 7, 'keywords': ['PL/pgSQL', 'процедура', 'исключения']}, page_content="CREATE OR REPLACE PROCEDURE cs_create_job(v_job_id integer) AS $$ DECLARE a_running_job_count integer; BEGIN LOCK TABLE cs_jobs IN EXCLUSIVE MODE; SELECT count(*) INTO a_running_job_count FROM cs_jobs WHERE end_stamp IS NULL; IF a_running_job_count > 0 THEN COMMIT; RAISE EXCEPTION 'Unable to create a new job: a job is currently running'; END IF; DELETE FROM cs_active_job; INSERT INTO cs_active_job(job_id) VALUES (v_job_id); BEGIN INSERT INTO cs_jobs (job_id, start_stamp) VALUES (v_job_id, now()); EXCEPTION WHEN unique_violation THEN NULL; END; COMMIT; END; $$ LANGUAGE plpgsql;")

In [23]:
model_name = "DeepPavlov/rubert-base-cased"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
embedding = HuggingFaceEmbeddings(model_name=model_name,
                                  model_kwargs=model_kwargs,
                                  encode_kwargs=encode_kwargs)

vector_store = FAISS.from_documents(documents, embedding=embedding)

No sentence-transformers model found with name DeepPavlov/rubert-base-cased. Creating a new one with mean pooling.
Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model

In [24]:
embedding_retriever = vector_store.as_retriever(search_kwargs={"k": 5})

In [25]:
llm = GigaChat(credentials=API_KEY,
              model=MODEL,
              scope=SCOPE,
              verify_ssl_certs=IS_VERIFY_SSL_CERTS,
              profanity_check=IS_PROFANITY_CHECK)

In [26]:
prompt="""
Контекст: {context}
Вопрос:
Ниже представлен код запроса для СУБД Oracle. Преобразуй этот код для его использования в PostgreSQL, строго соблюдая следующие требования:

Перевод синтаксиса и функций: Если в Oracle используются специфические функции или операторы, замени их на точные эквиваленты в PostgreSQL. Например, для SEQUENCE, используй SERIAL или GENERATED. Обязательно комментируй каждую замену и поясняй, почему она сделана.
Адаптация типов данных: Внимательно следи за типами данных, которые могут отличаться между Oracle и PostgreSQL (например, VARCHAR2 в Oracle и VARCHAR в PostgreSQL). Перечисли все измененные типы данных и объясни, почему они были изменены.
Поддержка полной функциональности: Сохрани логику запроса без изменений. Не удаляй и не модифицируй части кода, которые не требуют преобразования.
Проверка совместимости: Убедись, что финальный код полностью соответствует стандартам PostgreSQL и корректно выполняется в этой СУБД. Если существуют ограничения, которые могут повлиять на выполнение, обязательно укажи их.
Пояснение работы запроса: Помимо преобразования, опиши в деталях, что делает этот запрос. Объясни шаги логики, чтобы было ясно, как работает запрос на уровне действий (выборка, фильтрация, агрегация и т.д.).
Присылание кода в sql Присылай только полный код, не нужно присылать его частями.
Использование EXCEPT Не используй MINUS, мне нужно через EXCEPT.
Конструкция SQL: Используй конструкцию SQL только для того, чтобы отправить весь код целиком. Для отдельных пояснений частей кода конструкцию SQL применять категорически запрещено, иначе система перестанет работать и я буду за это наказан.
Напиши обязательно описание переданного запроса. Обязательно!!!
Если в запросе есть CREATE TABLE или INSERT обязательно оставь их, у меня нет пальцев.
{input}
"""

In [27]:
prompt = ChatPromptTemplate.from_template(prompt)

In [29]:
document_chain = create_stuff_documents_chain(
    llm=llm,
    prompt=prompt
)

In [30]:
retrieval_chain = create_retrieval_chain(embedding_retriever, document_chain)

# Start

In [31]:
with open("sql_code.sql", "r") as file:
    oracle_code = file.read()
print(oracle_code)

create table my_brick_collection (
  colour varchar2(10),
  shape  varchar2(10),
  weight integer
);

create table your_brick_collection (
  height integer,
  width  integer,
  depth  integer,
  colour varchar2(10),
  shape  varchar2(10)
);

insert into my_brick_collection values ( 'red', 'cube', 10 );
insert into my_brick_collection values ( 'blue', 'cuboid', 8 );
insert into my_brick_collection values ( 'green', 'pyramid', 20 );
insert into my_brick_collection values ( 'green', 'pyramid', 20 );
insert into my_brick_collection values ( null, 'cuboid', 20 );

insert into your_brick_collection values ( 2, 2, 2, 'red', 'cube' );
insert into your_brick_collection values ( 2, 2, 2, 'blue', 'cube' );
insert into your_brick_collection values ( 2, 2, 8, null, 'cuboid' );

commit;

select colour, shape from your_brick_collection
minus
select colour, shape from my_brick_collection;


In [32]:
print(prompt)

input_variables=['context', 'input'] messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'input'], template='\nКонтекст: {context}\nВопрос:\nНиже представлен код запроса для СУБД Oracle. Преобразуй этот код для его использования в PostgreSQL, строго соблюдая следующие требования:\n\n1. **Перевод синтаксиса и функций:** Если в Oracle используются специфические функции или операторы, замени их на точные эквиваленты в PostgreSQL. Например, для `SEQUENCE`, используй `SERIAL` или `GENERATED`. Обязательно комментируй каждую замену и поясняй, почему она сделана.\n2. **Адаптация типов данных: Внимательно следи за типами данных, которые могут отличаться между Oracle и PostgreSQL (например, `VARCHAR2` в Oracle и `VARCHAR` в PostgreSQL). Перечисли все измененные типы данных и объясни, почему они были изменены.\n3. **Поддержка полной функциональности:** Сохрани логику запроса без изменений. Не удаляй и не модифицируй части кода, которые не требуют преобразования.

In [ ]:
resp1 = retrieval_chain.invoke(
    {"input": oracle_code}
)

In [323]:
print(resp1["answer"])

Для выполнения задачи перевода SQL-запроса с Oracle на PostgreSQL, необходимо учесть различия в синтаксисе и функциях этих систем управления базами данных. Вот подробное описание каждого шага преобразования:

```sql
-- Создание таблицы my_brick_collection в PostgreSQL
CREATE TABLE my_brick_collection (
  colour VARCHAR(10),
  shape  VARCHAR(10),
  weight INTEGER
);

-- Создание таблицы your_brick_collection в PostgreSQL
CREATE TABLE your_brick_collection (
  height INTEGER,
  width  INTEGER,
  depth  INTEGER,
  colour VARCHAR(10),
  shape  VARCHAR(10)
);

-- Вставка данных в таблицу my_brick_collection
INSERT INTO my_brick_collection VALUES ('red', 'cube', 10);
INSERT INTO my_brick_collection VALUES ('blue', 'cuboid', 8);
INSERT INTO my_brick_collection VALUES ('green', 'pyramid', 20);
INSERT INTO my_brick_collection VALUES ('green', 'pyramid', 20);
INSERT INTO my_brick_collection VALUES (NULL, 'cuboid', 20);

-- Вставка данных в таблицу your_brick_collection
INSERT INTO your_brick_col